1.Load the necessary libraries

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  
import torchvision.datasets as dataset  
import torchvision.transforms as transforms  
import torchvision.utils as utils 
import torch.utils.data as data_utils  

: 

2.Defining hyperparameters

In [ ]:
BATCH_SIZE = 3
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 10

: 

3.Build a pipeline and process the image

In [ ]:
pipeline = transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize  ((0.1307,),(0.3081,))
    
])

: 

4. Downloading and loading data

In [ ]:
from torch.utils.data import DataLoader

train_data = datasets.CIFAR10('data',train=True,download=True,transform=pipeline)
test_data =datasets.CIFAR10('data',train=False,download=True,transform=pipeline)

train_dataloader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=BATCH_SIZE)

: 

In [ ]:
import numpy as np             
import math                     
import matplotlib.pyplot as plt 

print("Original Pytorch image")
image, label = train_data[2]
print("torch image shape:", image.shape)
print("torch image label:", label)
 
print("\nNumpy image after channel conversion")
image = image.numpy().transpose(1,2,0)  #交换维度，从GBR换成RGB
print("numpy image shape:", image.shape)
print("numpy image label:", label)
 
plt.imshow(image)
plt.show()


: 

5. Building a network model

In [ ]:
class Mymodel(nn.Module):
    def __init__(self):
        super(Mymodel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.model(x)
        return x

        output = F.log_softmax(x,dim=1)

        return output


: 

In [ ]:
'''
class Mymodel(nn.Module):
  def __init__(self):
    super(Mymodel,self).__init__()
    self.conv1 = nn.Conv2d(3, 32, 5, 1, 2)
    self.conv2 = nn.Conv2d(32, 32, 5, 1, 2)
    self.fc1 = nn.Linear(64 * 4 * 4, 64)
    self.fc2 = nn.Linear(64, 10)

  def forward(self,x):
    input_size = x.size(0)
    x = self.conv1(x)
    x = F.relu(x)
    

    x = self.conv2(x)
    x = F.relu(x)

    x = x.view(input_size, -1)

    x = self.fc1(x)
    x = F.relu(x)

    x = self.fc2(x)

    output = F.log_softmax(x,dim=1)

    return output
'''


: 

6. Setting up the optimiser

In [ ]:
model = Mymodel().to(DEVICE)

optimizer = optim.Adam(model.parameters())

: 

7. Setting up training methods

In [ ]:
def train_model(model,device,train_dataloader,optimizer,epoch):
    model.train()
    for batch_index,(data,target) in enumerate(train_dataloader):
        
        data,target = data.to(device),target.to(device)
        
        optimizer.zero_grad()

        output = model(data)

        loss = F.cross_entropy(output,target)

        loss.backward()

        optimizer.step()
        
        if batch_index % 3000 == 0:
            print("Train Epoch : {} \t Loss : {:.6f}".format(epoch,loss.item()))

: 

8. Setting up the test method

In [ ]:
def test_model(model,device,test_dataloader):
    model.eval()

    correct = 0.0
    test_loss = 0.0

    with torch.no_grad():
        for data,target in test_dataloader:
            data,target = data.to(device),target.to(device)

            output = model(data)

            test_loss += F.cross_entropy(output,target)

            pred = output.argmax(dim=1)

            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss/= len(test_dataloader.dataset)

        print("Test_average loss:{:.4f},Accuracy:{:.3F}\n".format(test_loss.item(),100.0*correct/len(test_dataloader.dataset)))



: 

9. Calling methods

In [ ]:
for epoch in range(1,EPOCHS + 1):
      train_model(model,DEVICE,train_dataloader,optimizer,epoch)
      test_model(model,DEVICE,test_dataloader)

: 